## 1.1 Методы репроцессинга данных и выделение значиммых фич

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_excel("data/data.xlsx")
df.columns=["id", "no", "date", "origin_date", "recipient", "author", "content", "extra"]
df.set_index('id', inplace=True)
df.drop(df.index[:1], inplace=True)
df.head()

,no,date,origin_date,recipient,author,content,extra
id,,,,,,,
2,WSR/1-837/2019,2019-04-09,05-338\n09.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,NaN
3,WSR/1-835/2019,2019-04-09,И10-14/3137\n09.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,NaN
4,WSR/1-834/2019,2019-04-09,Д2/7121-ИС\n03.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,NaN
5,WSR/1-833/2019,2019-04-09,15-20/06-911\n08.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,NaN
6,WSR/1-831/2019,2019-04-09,Исх01/0403\n02.04.2019,Миронова С.В.,Островский А.В. (Губернатор Смоленской области),Вх - Об участии в Чемпионате мира по профессио...,NaN


Исходя из предметной области, для классификации корреспонденции будущей модели не нужны будут такие данные как дата отправления и создания документа, а также его номер в системе. Поэтому оставим только имя получателя письма (будущую целевую переменную), отправителя, краткое содержание, а также приложения к письмам, т.к. из них в будущем необходимо будет извлечь дополнительную информацию

In [3]:
df=df.drop(["no","date","origin_date"], axis=1)
df.shape

(4966, 4)

## 1.2 Сегментация композитных фич датасета
Сложными атрибутами в данном наборе данных являются значения всех полей, кроме целевого, поэтому нам необходимо будет обработать в ходе сессии все. В разделе 1.2 будет обработано поле получателей, в разделе 1.3 поле отправителей, в разделе 1.4 частично будет затронуто поле содержания писем, а в 1.5 - поле приложений к письмам.

In [4]:
df["recipient"].value_counts()

Глушко Д.Е.                        1559
Уразов Р.Н.                        1466
Крайчинская С.Б.                    315
Лысова С.И.                         234
Тымчиков А.Ю.                       229
                                   ... 
Уразов Р.Н.\nНедоспасова О.В.         1
Недоспасова О.В.\nМитяева И.В.        1
Домаева О.Ю.                          1
Глушко Д.Е.\nКожевникова Е.И.         1
Мухаметзянов Д.Ф.\nСудаков К.Ю.       1
Name: recipient, Length: 161, dtype: int64

Поле хранит в себе конкатенированые через спецсимвол "\n" значения. Разделим их и преобразуем имена в более легко интерпретируемый с помощью кода вид (из "Фамилия И.О." в "ФамилияИО") и применим метод CountVectorizer библиотеки Scikit-learn для получения схожей с One-Hot матрицы, в которой столбцы - все варианты адресатов, а значения в строках - 0 (не адресат) и 1 (адресат)

In [5]:
def processRecs(inp):
    inp=inp.replace(' ','')
    inp=inp.replace('.','')
    inp=inp.replace('\n',' ')
    return inp

df_r=df['recipient'].apply(processRecs)
df_r

id
2         УразовРН
3         УразовРН
4         ИванюкЛА
5         УразовРН
6       МироноваСВ
           ...    
4963      УразовРН
4964      УразовРН
4965      УразовРН
4966      УразовРН
4967      УразовРН
Name: recipient, Length: 4966, dtype: object

In [6]:
from sklearn.feature_extraction.text import CountVectorizer as CV
CVr = CV()
rec_vecs=CVr.fit_transform(df_r)
CVr.get_feature_names()
# Получившиеся признаки

['абдулгалеевзт',
 'абдулганиевфс',
 'абдуллазяновэю',
 'абдуллинам',
 'аблязовка',
 'абязоваюа',
 'агеевшр',
 'акимовпм',
 'алешинав',
 'алимоваа',
 'антипинса',
 'антоновюв',
 'артемоватв',
 'афанасьеваи',
 'афанасьевасн',
 'афанасьевмп',
 'ахметовмг',
 'ахметшинаи',
 'ахметшинас',
 'ахметшинрк',
 'аштаевакд',
 'аюповаих',
 'багровюн',
 'бадретдиновбм',
 'базероэ',
 'баковаюв',
 'балтусоваоа',
 'бариевии',
 'бариевмм',
 'барышеватг',
 'бастрыкинаи',
 'безносикована',
 'богучарскаяас',
 'большаковав',
 'бородинис',
 'бортниковав',
 'булавинви',
 'бургановрт',
 'вазыховна',
 'валеевдр',
 'валитовиа',
 'валиуллинаа',
 'валиуллинрн',
 'васильеваою',
 'вахитоврр',
 'вишняковаоб',
 'волковаи',
 'выборноваа',
 'габдрашитовбр',
 'габдурахмановлр',
 'газизовах',
 'гайдаевми',
 'гайзатуллинрр',
 'гайнутдиновар',
 'галеевэг',
 'галиуллинрф',
 'гараеваар',
 'гараевзф',
 'гариповим',
 'гатаулинтм',
 'гатауллинаэ',
 'гатиятуллинрх',
 'гафаровмр',
 'гафуровир',
 'гаязовзф',
 'гилмановкк',
 'гильмут

In [7]:
encoded_rec=pd.DataFrame(rec_vecs.toarray())
encoded_rec.columns=CVr.get_feature_names()
encoded_rec

,абдулгалеевзт,абдулганиевфс,абдуллазяновэю,абдуллинам,аблязовка,абязоваюа,агеевшр,акимовпм,алешинав,алимоваа,...,шакировра,шафигуллинлн,шаяхметоврк,шигабутдиновак,шойгуск,щугоревна,юсуповиу,юшкосв,ягфароваию,якубовюд
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1.3 Дополнение датасета недостающими данными
В данных об отправителе указано как лицо, являющееся отправителем, так и организация, которую оно представляет. Разбиение этого поля на два даст больше информации об отправителе, а также впоследствии поможет выявить особо важные лица (будут встречены чаще других)

In [8]:
df['author'].value_counts()

Черноскутова И.А. (Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО)    93
Картошкин С.А. (Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО)       41
Потехина И.П. (Министерство образования и науки Российской Федерации)                                                                                      39
Шевелева А.А. (Министерство образования Тульской области)                                                                                                  38
Соляников Ю.В. (Комитет по образованию Правительства Санкт-Петербурга)                                                                                     37
                                                                                                                                                           ..
Ефимов С.Ю. (43. Министерство промышленности и торго

Для этого определим две функции: первая возвращает данные внутри скобок, а вторая - данные до скобок, а после - сохраним обработанный обеими функциями столбец в две различных переменных

In [9]:
def getBracketContent(inp):
    pos=inp.find('(')
    if (pos==-1):
        return inp
    return inp[pos+1:len(inp)-1]

def getBeforeBracket(inp):
    pos=inp.find('(')
    if (pos==-1):
        return inp
    if (inp.find('инистерство')==-1):
        return inp[:pos-1]
    return None

In [10]:
authors=df['author'].apply(getBeforeBracket)
authors.value_counts()

Губайдуллин Э.Ф.    49
Соляников Ю.В.      45
Шохин А.            27
Артемьев И.А.       25
Волощук Л.В.        25
                    ..
Малькевич А.А.       1
Харлашкин С.В.       1
Мутко В.Л.           1
Шанаурова И.Ш.       1
Сумбатян М.С.        1
Name: author, Length: 1515, dtype: int64

In [11]:
orgs=df['author'].apply(getBracketContent)
orgs.value_counts()

Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО    147
Министерство образования и науки Российской Федерации                                                                                   82
Министерство просвещения Российской Федерации                                                                                           64
Аппарат Правительства Российской Федерации                                                                                              60
Министерство образования Тульской области                                                                                               52
                                                                                                                                      ... 
Правительство Оренбургской области                                                                                                       1
Правительство Ханты-Мансийс

## 1.4 Формирование словарей данных
В данном разделе нам необходимо построить на основе кратких сведений о содержании писем построить словарь, содержащий в себе слова, которые могут быть использованы для предсказания target-переменной в дальнейшем, а также общее количество вхождений слов в документы.

In [12]:
content=df['content']
vectorizer=CV(stop_words=['00', 'г', 'гг', 'о', 'об', 'по', 'для', 'на', 'из', '0001', '001', '0020', '00321', '00366', '006', '01', '02', '03', '04', '043', '05', '05пр', '06', '062', '06g', '06пр', '07', '073', '074', '08', '09', '0900д', '10', '100', '1030', '104', '10584', '1065', '11', '1187', '119', '12', '120', '125', '1265', '13', '137', '14', '140', '141', '1426', '1455', '147', '1497', '15', '1551', '16', '17', '170315019', '170526050', '1723', '178', '17г', '18', '1849', '188', '189', '19', '1921', '194', '1942', '1968', '1f3', '1б', '1в', '1г', '1го', '1ж', '1с', '20', '200', '20118', '2015', '2016', '2016г', '2017', '2017г', '2017гг', '2018', '20185г', '2018г', '2019', '2019г', '2019гг', '202', '2020', '2021', '2021г', '2021гг', '2022', '2024', '2024г', '2025', '2030', '2035', '2073', '2075', '209', '20пр', '21', '210', '216', '2160', '217', '218', '22', '22017', '2220', '2225', '2249', '226', '228', '23', '230', '2323', '234', '2343', '24', '2429', '2494', '25', '2550', '2582', '2585', '26', '2603', '2604', '2605', '2606', '261', '262', '263', '265', '2679', '27', '273', '278', '279', '27февраля', '28', '286', '289', '29', '295', '298', '299', '2996818', '2а', '2б', '30', '300', '300579', '301', '30107', '306', '30пр', '31', '3100', '32', '321гс', '3259', '326', '328', '33', '332', '3372', '33с', '342', '349', '35', '3510', '36', '37', '372', '3751р', '37680', '38', '3857', '39', '3995', '3d', '3dld', '3и12', '3х', '404', '4060', '408', '42', '423', '425', '42548', '436', '437', '438', '45', '456', '46', '4600037832', '47', '486', '49', '4919п', '496', '50', '5000', '506', '51', '52', '52пр', '53', '58', '580', '58199', '59657', '598', '60', '61884', '625', '63', '63611', '638', '64392', '65', '652', '65935', '6629', '671', '677', '69', '690', '694', '695', '6981', '701', '708', '709', '70пр', '710', '7115', '73', '7412', '7493', '75', '750', '762', '77', '770400207282', '7721', '791', '807', '83', '839', '84', '846', '85', '869', '88', '89', '901', '91', '911', '9504', '976', '____', '__________', '____________', '_____________', '________________', '_________________', '____________________', '______________________', '________________________', '_________________________', '__________________________', '_____________________________', '___________________________________', '_____________________________________', '_______________________________________', '____________________________________________', '_____________________________________________', '_о', 'about'])
vectors=vectorizer.fit_transform(content)
vectorizer.vocabulary_

{'вх': 658,
 'участии': 4697,
 'совещании': 4101,
 'вопросу': 597,
 'доработки': 1028,
 'фэо': 4831,
 'предоставления': 3262,
 'субсидии': 4361,
 'реализацию': 3748,
 'проекта': 3482,
 'билет': 355,
 'будущее': 397,
 'апр': 245,
 'командировании': 1645,
 'саликовой': 3906,
 'глушко': 760,
 'участия': 4700,
 'заседании': 1221,
 'оргкомитета': 2612,
 'проведения': 3439,
 'фнч': 4800,
 'кемеровской': 1582,
 'области': 2401,
 'чемпионате': 4926,
 'мира': 2090,
 'профессиональному': 3567,
 'мастерству': 1952,
 'стандартам': 4296,
 'ворлдскиллс': 601,
 'worldskills': 104,
 'kazan': 60,
 'церемонии': 4883,
 'открытия': 2709,
 'вузовского': 655,
 'чемпионата': 4923,
 'ставрополь': 4288,
 'направлении': 2219,
 'протокола': 3546,
 'ходе': 4851,
 'реализации': 3747,
 'национального': 2257,
 'демография': 903,
 'субъектах': 4368,
 'россии': 3854,
 'согласовании': 4127,
 'дат': 856,
 'национальном': 2259,
 'проведении': 3437,
 'заседания': 1225,
 'министров': 2071,
 'образования': 2434,
 'государст

In [13]:
vecs_df=pd.DataFrame(vectors.toarray())
vecs_df.columns=vectorizer.get_feature_names()
vecs_df

,academy,airlung,amberforum,arena,articskills,atomskills,autodesk,budapest,cad,cd,...,ялтинском,ялтинскому,ямедовой,янао,янв,январе,января,янтарной,ярославль,ярославская
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1.5 Преобразование переадресаций
При переадресации в исходных данных в поле "приложения" указывается лицо, которому было переадресовано письмо. Эту информацию необходимо вычленить. Проблему в данном случае составляет наличие не нужных в рамках нашей задачи приложений, а также нестандартизированность записей в этом поле. Для того, чтобы устранить эту проблему, необходимо провести обработку этого поля.

Создадим метод, определяющий записи о переадресации по наличию в них слов "передан" и "направлено", а также стирающий информацию о дате, и применим его к столбцу с приложениями

In [14]:
def filterExtras(inp):
    if (str(inp).lower().find('направлено')!=-1 or str(inp).lower().find('передан')!=-1):
        # Некоторые записи содержат ключевые слова, но не являются сведениями о переадресации конкретному лицу
        #Следующее условие их отфильтрует:
        if (str(inp.lower()).find('@')==-1 and str(inp.lower()).find('департамент')==-1 and str(inp.lower()).find('бухгалт')==-1 and str(inp.lower()).find('отдел')==-1 and str(inp).lower().find('академ')==-1 and str(inp).lower().find('адресат')==-1 and str(inp.lower()).find('суд')==-1):
            inp=inp.replace('1','')
            inp=inp.replace('2','')
            inp=inp.replace('3','')
            inp=inp.replace('4','')
            inp=inp.replace('5','')
            inp=inp.replace('6','')
            inp=inp.replace('7','')
            inp=inp.replace('8','')
            inp=inp.replace('9','')
            inp=inp.replace('0','')
            inp=inp.replace('- ','')
            inp=inp.replace('-','')
            inp=inp.replace('.. ','')
            inp=inp.replace(' ..','')
            inp=inp.replace('..','')
            inp=inp.replace('\n',' ')
            return inp
    return None

Посмотрим, какие значения остались в этом столбце после частичной обработки:

In [15]:
df['extra']=df['extra'].apply(filterExtras)
df['extra'].value_counts()

Направлено Митькиной А.                            10
Оригиналы приложений переданы Смирновой Л.          5
Направлено Мироновой С.                             4
Документы сшиты. Переданы Гайдаеву М.               3
Оригиналы приложений переданы Туниной К.            2
                                                   ..
Направлено Орловой, Иванюк и Дмитриеву              1
Заявка передана лично в руки И. Зайчиковой          1
Передано Митькиной А., т.к. ответ на ее письмо.     1
оригиналы переданы Константиновой А.                1
Передан Карпуниной А.                               1
Name: extra, Length: 67, dtype: int64

## 1.6 Формирование пресетов

In [16]:
data2=pd.concat([df, pd.DataFrame(encoded_rec), pd.DataFrame(authors), pd.DataFrame(orgs), pd.DataFrame(vecs_df)], axis=1)

In [17]:
data2.to_csv("data/data2.csv", sep=",")
data2.head()

,recipient,author,content,extra,абдулгалеевзт,абдулганиевфс,абдуллазяновэю,абдуллинам,аблязовка,абязоваюа,...,ялтинском,ялтинскому,ямедовой,янао,янв,январе,января,янтарной,ярославль,ярославская
0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,None,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,None,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,None,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pd.concat([pd.DataFrame(encoded_rec), pd.DataFrame(authors), pd.DataFrame(orgs), pd.DataFrame(vecs_df)], axis=1).to_csv("data/data_clear.csv", sep=",")